In [7]:
import os
import sys
sys.path.append("C:\pruning_techniqes\cnn_pruning_cifar10-1\models")
import numpy as np
import pandas as pd

import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint

# Import your ResNet-20 model
from resnet20 import build_resnet20  # Make sure resnet20_baseline.py is in the same folder or sys.path

In [8]:

def lr_schedule(epoch):
    if epoch < 80:
        return 0.1
    elif epoch < 120:
        return 0.01
    else:
        return 0.001

In [9]:
# Load CIFAR-10
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

In [10]:
# Normalize
mean = x_train.mean(axis=(0, 1, 2), keepdims=True)
std = x_train.std(axis=(0, 1, 2), keepdims=True)
x_train = (x_train - mean) / std
x_test = (x_test - mean) / std

y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# Data Augmentation
datagen = ImageDataGenerator(horizontal_flip=True,
                             width_shift_range=0.1,
                             height_shift_range=0.1)
datagen.fit(x_train)

In [11]:
# Build & Compile Model
model = build_resnet20()
model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9, nesterov=True),
              metrics=["accuracy"])

In [12]:
# Callbacks
checkpoint = ModelCheckpoint(
    filepath=r"C:\pruning_techniqes\cnn_pruning_cifar10-1\results\baseline_resnet20.h5",
    save_best_only=True,
    monitor="val_accuracy",
    mode="max"
)
lr_scheduler = LearningRateScheduler(lr_schedule)

In [13]:
# Train
model.fit(datagen.flow(x_train, y_train, batch_size=128),
          epochs=200,
          validation_data=(x_test, y_test),
          callbacks=[checkpoint, lr_scheduler])

Epoch 1/200
391/391 [==============================] - 32s 77ms/step - loss: 1.8194 - accuracy: 0.3981 - val_loss: 2.0319 - val_accuracy: 0.4045 - lr: 0.1000
Epoch 2/200
391/391 [==============================] - 29s 74ms/step - loss: 1.3749 - accuracy: 0.5615 - val_loss: 2.2479 - val_accuracy: 0.4428 - lr: 0.1000
Epoch 3/200
391/391 [==============================] - 29s 74ms/step - loss: 1.1278 - accuracy: 0.6574 - val_loss: 1.4208 - val_accuracy: 0.5986 - lr: 0.1000
Epoch 4/200
391/391 [==============================] - 29s 73ms/step - loss: 0.9818 - accuracy: 0.7124 - val_loss: 1.0644 - val_accuracy: 0.6922 - lr: 0.1000
Epoch 5/200
391/391 [==============================] - 29s 73ms/step - loss: 0.8936 - accuracy: 0.7471 - val_loss: 1.3900 - val_accuracy: 0.6352 - lr: 0.1000
Epoch 6/200
391/391 [==============================] - 29s 73ms/step - loss: 0.8320 - accuracy: 0.7706 - val_loss: 1.7184 - val_accuracy: 0.5590 - lr: 0.1000
Epoch 7/200
391/391 [==============================]